In [2]:
import numpy as np
from pycocotools.coco import COCO
import time
import pickle
import json

from config import ms_coco_dir

In [2]:
# use fc6 descriptors

# fc6_train = np.load(ms_coco_dir + '/descriptors/fc6/train.npy', encoding='latin1').item()
# fc6_val = np.load(ms_coco_dir + '/descriptors/fc6/val.npy', encoding='latin1').item()
# fc6_test = np.load(ms_coco_dir + '/descriptors/fc6/test.npy', encoding='latin1').item()

# fc6 = dict(fc6_train)
# fc6.update(fc6_val)
# fc6.update(fc6_test)

# descriptors_train = descriptors_val = fc6

In [3]:
# use xception descriptors

xc_train = pickle.load(open(ms_coco_dir + '/descriptors/xception/xception-train2017-descriptors.pic', 'rb'))
xc_val = pickle.load(open(ms_coco_dir + '/descriptors/xception/xception-val2017-descriptors.pic', 'rb'))

descriptors_train = xc_train
descriptors_val = xc_val

In [5]:
# normalize to norm_2 = 1
normalized_descriptors_train = {k : v/np.linalg.norm(v) for k,v in descriptors_train.items()}
normalized_descriptors_val   = {k : v/np.linalg.norm(v) for k,v in descriptors_val.items()}

In [6]:
annFileTrain = '{}/annotations/captions_train2017.json'.format(ms_coco_dir)
annFileVal = '{}/annotations/captions_val2017.json'.format(ms_coco_dir)
cocoTrain = COCO(annFileTrain)
cocoVal = COCO(annFileVal)

loading annotations into memory...
Done (t=2.86s)
creating index...
index created!
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!


In [8]:
trainIds = [i for i in cocoTrain.getImgIds() if i in descriptors_train]
valIds   = [i for i in cocoVal.getImgIds() if i in descriptors_val]

In [9]:
len(trainIds), len(valIds)

(118287, 5000)

In [14]:
def eucleidean_dist(x, y):  # not really an eucleidean distance, more like mean-squared distance
    z = x - y
    return np.dot(z, z)    

def cosine_dist(x, y):
    return 1 - np.dot(x,y)**2 / np.dot(x,x) / np.dot(y,y)

def cosine_dist2(x, y):  # supposes x and y were normalized to norm = 1
    return 1 - np.dot(x,y)

def knn(v_id):
    '''return t_id from train set that is closest to v_id from validation set'''
    min_d = np.infty
    min_id = None
    for t_id in trainIds:
        d = cosine_dist2(normalized_descriptors_train[t_id], normalized_descriptors_val[v_id])
#         d = eucleidean_dist(descriptors_train[t_id], descriptors_val[v_id])
        if d < min_d:
            min_d = d
            min_id = t_id
    return min_id    

def predict_val():
    predictions = {}
    t_start = time.clock()
    for i, v_id in enumerate(valIds):
        print('Predicting {}/{}'.format(i+1, len(valIds)))
        t_id = knn(v_id)
        if t_id is None:
            continue
        annIds = cocoTrain.getAnnIds(imgIds=[t_id])
        anns = cocoTrain.loadAnns(annIds)[0] # take the first caption
        predictions[v_id] = anns['caption']
        
        v_annIds = cocoVal.getAnnIds(imgIds=[v_id])
        v_anns = cocoVal.loadAnns(v_annIds)[0]
        
        print('Predicted:', anns['caption'])
        print('True:     ', v_anns['caption'])
        
        t_now = time.clock()
        t_elapsed = t_now - t_start
        str_elapsed = time.strftime('%Hh %Mm %Ss', time.gmtime(t_elapsed))
        t_eta = t_elapsed / (i+1) * (len(valIds)-i-1)
        str_eta = time.strftime('%Hh %Mm %Ss', time.gmtime(t_eta))
        print('elapsed:', str_elapsed, '. ETA:', str_eta)
    return predictions

In [15]:
# %load_ext line_profiler
# %lprun -f knn predict_val()
predictions = predict_val()

1/5000
Predicted: some people are standing together around some food
True:      A man is in a kitchen making pizzas.
elapsed: 00h 00m 01s . ETA: 01h 41m 06s
2/5000
Predicted: A luxurious bathroom with a towel set out.
True:      The dining table near the kitchen has a bowl of fruit on it.
elapsed: 00h 00m 02s . ETA: 01h 39m 56s
3/5000
Predicted: A kid holding a video game controller in his hand.
True:      a person with a shopping cart on a city street 
elapsed: 00h 00m 03s . ETA: 01h 38m 43s
4/5000
Predicted: A lady on a pink bike wearing a blue dress and pink boots.
True:      A person on a skateboard and bike at a skate park.
elapsed: 00h 00m 04s . ETA: 01h 37m 45s
5/5000
Predicted: A man riding his bicycle down the road.
True:      a blue bike parked on a side walk 
elapsed: 00h 00m 05s . ETA: 01h 36m 42s
6/5000
Predicted: A bathroom with a toiled and a bathtub
True:      A bathroom that has a broken wall in the shower.
elapsed: 00h 00m 06s . ETA: 01h 36m 01s
7/5000
Predicted: A ba

Predicted: Looking at a barge cross a channel of water under a cloudy sky
True:      there are two large boats that are in the water 
elapsed: 00h 00m 54s . ETA: 01h 32m 53s
49/5000
Predicted: People standing and watching an elephant, while the elephant puts his trunk on a ladies head.
True:      An elephant standing under the shade of a tree.
elapsed: 00h 00m 55s . ETA: 01h 32m 51s
50/5000
Predicted: Man on side of street with umbrella raised from rain
True:      A black and white image with a colored british flag umbrella
elapsed: 00h 00m 56s . ETA: 01h 32m 49s
51/5000
Predicted: An elephant stands in the grass staring out.
True:      There is an elephant standing near a tree.
elapsed: 00h 00m 57s . ETA: 01h 32m 46s
52/5000
Predicted: A man walking across a street in the rain.
True:      THERE ARE WOMEN THAT ARE LAUGHING UNDER THE UMBRELLA
elapsed: 00h 00m 58s . ETA: 01h 32m 43s
53/5000
Predicted: A couple of Wii remotes and the Mario designed Wii system. 
True:      Several middle e

Predicted: A bed frame made out of wood and tree branches.
True:      A picture of a recently married couple displayed behind glass.
elapsed: 00h 01m 46s . ETA: 01h 31m 58s
96/5000
Predicted: A store window with a lot antiques on display.
True:      A man in bowler hat and lab coat by people in a tv frame.
elapsed: 00h 01m 48s . ETA: 01h 31m 59s
97/5000
Predicted: The young girl joyfully bites into her slice of pizza.
True:      A young girl eating a slice of pizza.
elapsed: 00h 01m 49s . ETA: 01h 31m 58s
98/5000
Predicted: A bed frame made out of wood and tree branches.
True:      A chair sitting in the middle of the room, in a black and white photo.
elapsed: 00h 01m 50s . ETA: 01h 31m 56s
99/5000
Predicted: a pizza with a lot of cheese and some meat and veggies 
True:      A slice of pizza that is on top of a napkin.
elapsed: 00h 01m 51s . ETA: 01h 31m 59s
100/5000
Predicted: a man swinging a tennis racquet at a tennis ball
True:      A tennis player hits a high ball in front of an a

Predicted: The bath room of a hotel room with the shower over the tub
True:      A white toilet and cup in a room.
elapsed: 00h 02m 40s . ETA: 01h 31m 45s
143/5000
Predicted: A piece of pie and a fork sit on a white saucer.
True:      A fork rests on a plate next to a piece of cake.
elapsed: 00h 02m 42s . ETA: 01h 31m 43s
144/5000
Predicted: A computer, snacks, and family pictures sit on a desk.
True:      Several different pieces of electronic equipment and computers on a desk.
elapsed: 00h 02m 43s . ETA: 01h 31m 41s
145/5000
Predicted: A Super Bowl party with lots of pizza and drinks.
True:      A man sitting at a table in front of a plate of food.
elapsed: 00h 02m 44s . ETA: 01h 31m 40s
146/5000
Predicted: View from under a large plane flying in the sky.
True:      An Air Force jet flying in a deep blue sky.
elapsed: 00h 02m 45s . ETA: 01h 31m 39s
147/5000
Predicted: A zebra that is standing in the grass.
True:      A field with various wildlife and trees in the background.
elapsed:

Predicted: A bookshelf with a bear toy and various books.
True:      A row of white teddy bears on shelf next to DVDs.
elapsed: 00h 03m 32s . ETA: 01h 30m 45s
189/5000
Predicted: A black bear is walking through the field
True:      A baby brown bear standing on top of a rock.
elapsed: 00h 03m 34s . ETA: 01h 30m 48s
190/5000
Predicted: a group of young people playing soccer in a field
True:      People walking on sand next to a mountainous area.
elapsed: 00h 03m 35s . ETA: 01h 30m 52s
191/5000
Predicted: there is a small dog sitting on the floor playing with a toy
True:      A small dog laying on a bed next to a laptop computer.
elapsed: 00h 03m 36s . ETA: 01h 30m 51s
192/5000
Predicted: A young man sitting on top of a metal and wood bench.
True:      Two men play around on a nature trail.
elapsed: 00h 03m 37s . ETA: 01h 30m 50s
193/5000
Predicted: A herd of zebras are grazing in the open field
True:      three zebras in a field near bushes 
elapsed: 00h 03m 38s . ETA: 01h 30m 48s
194/5

...

Predicted: a giraffe standing in the brush looking around
True:      A giraffe and a zebra checking each other out.
elapsed: 01h 28m 27s . ETA: 00h 03m 36s
4805/5000
Predicted: A park bench with a blanket and cup of coffee.
True:      A man and his dog sit at the park.
elapsed: 01h 28m 28s . ETA: 00h 03m 35s
4806/5000
Predicted: A row of transit buses sitting in a parking lot.
True:      A street filled with buses and a white van.
elapsed: 01h 28m 29s . ETA: 00h 03m 34s
4807/5000
Predicted: Four giraffes huddling around a tree inside a fenced area.
True:      A group of giraffe standing next to each other on a dirt field.
elapsed: 01h 28m 30s . ETA: 00h 03m 33s
4808/5000
Predicted: A group of people standing next to each other.
True:      A giraffe stands alone outdoors at the zoo.
elapsed: 01h 28m 31s . ETA: 00h 03m 32s
4809/5000
Predicted: a close up of a giraffe drinking water
True:      A giraffe extends its tongue to drink water
elapsed: 01h 28m 32s . ETA: 00h 03m 31s
4810/5000
Pr

Predicted: A brown bear is ankle deep in some water.
True:      A brown bear walking on a ledge with trees in the background.
elapsed: 01h 29m 18s . ETA: 00h 02m 44s
4852/5000
Predicted: There is a polar bear standing next to a large boulder.
True:      A polar bear standing on a rock formation in an enclosure.
elapsed: 01h 29m 19s . ETA: 00h 02m 43s
4853/5000
Predicted: a vehicle got an accident and a police officer looking
True:      Zebras are grazing on grass by a car.
elapsed: 01h 29m 20s . ETA: 00h 02m 42s
4854/5000
Predicted: A group of zebra standing on top of a lush green field.
True:      A herd of zebra in a grass field.
elapsed: 01h 29m 21s . ETA: 00h 02m 41s
4855/5000
Predicted: a small zebra with black and white stripes
True:      A couple of zebra eating a small pile of hay.
elapsed: 01h 29m 23s . ETA: 00h 02m 40s
4856/5000
Predicted: The young girls are playing a game of soccer on the field. 
True:      a person playing frisbee on a field in sport wear
elapsed: 01h 29m 

Predicted: A woman with pink hair riding a motorcycle.
True:      Some people are enjoying themselves playing game. 
elapsed: 01h 30m 10s . ETA: 00h 01m 52s
4899/5000
Predicted: Small living room with a fireplace sitting in the middle of it. 
True:      A living room has a fireplace and a piano.
elapsed: 01h 30m 11s . ETA: 00h 01m 51s
4900/5000
Predicted: A man and woman playing a video game.
True:      A woman with large black tear drop earrings and a black dress looks at her cell phone. 
elapsed: 01h 30m 12s . ETA: 00h 01m 50s
4901/5000
Predicted: A pitcher in baseball throws the ball to the hitter.
True:      A group of men on a field playing baseball.
elapsed: 01h 30m 13s . ETA: 00h 01m 49s
4902/5000
Predicted: View of a baseball game from behind the catcher during a pitch.
True:      A man is swinging a bat a ball being pitched at him.
elapsed: 01h 30m 14s . ETA: 00h 01m 48s
4903/5000
Predicted: A person swinging at bat at a ball on a baseball field.
True:      A batter prepares t

Predicted: A female surfer carrying her surfboard on the beach.
True:      The woman sits on the curb writing on a notepad near a fire hydrant. 
elapsed: 01h 31m 00s . ETA: 00h 01m 01s
4945/5000
Predicted: A double decker bus driving down a street.
True:      A green and white bus on street next to dirt area.
elapsed: 01h 31m 01s . ETA: 00h 01m 00s
4946/5000
Predicted: A green and light blue bus traveling down a highway.
True:      A blue bus driving down a street past a park.
elapsed: 01h 31m 03s . ETA: 00h 00m 59s
4947/5000
Predicted: A giraffe at the zoo looks right at the camera 
True:      Two tall giraffes are next to bare trees.
elapsed: 01h 31m 04s . ETA: 00h 00m 58s
4948/5000
Predicted: A couple of giraffe walking through a zoo habitat.
True:      Two giraffes at the zoo during the day.
elapsed: 01h 31m 05s . ETA: 00h 00m 57s
4949/5000
Predicted: A duck swims along a large body of water.
True:      A large bird that is standing on some rocks by a river.
elapsed: 01h 31m 06s . 

Predicted: SUV and the trailer it is pulling utilizing street parking where there are parking meters.
True:      Two people carrying a kite to be flown later.
elapsed: 01h 31m 51s . ETA: 00h 00m 11s
4991/5000
Predicted: A boy in purple jersey swinging a baseball bat.
True:      A boy swinging a baseball bat at a game.
elapsed: 01h 31m 52s . ETA: 00h 00m 09s
4992/5000
Predicted: a black and white photo of a person riding a skate board on a hand rail
True:      A black and white image of a young man waiting to take his swing. 
elapsed: 01h 31m 53s . ETA: 00h 00m 08s
4993/5000
Predicted: Man about to hit the ball during a batting session.
True:      A young boy wearing a baseball uniform holding a baseball bat.
elapsed: 01h 31m 54s . ETA: 00h 00m 07s
4994/5000
Predicted: The girl in shorts is attempting to hit the tennis ball.
True:      A man playing tennis who is about to return a serve.
elapsed: 01h 31m 55s . ETA: 00h 00m 06s
4995/5000
Predicted: A blue teddy bear sitting on a pile of 

In [16]:
with open('predictions-knn-xception-cosine.pic', 'wb') as f:
    pickle.dump(predictions, f)

In [17]:
with open('predictions-knn-xception-cosine.pic', 'rb') as f:
    predictions = pickle.load(f)

In [18]:
predictions_list = [{'image_id' : iid, 'caption' : caption} for iid, caption in predictions.items()]
with open('predictions-knn-xception-cosine.json', 'w') as f:
    json.dump(predictions_list, f)